# Particle in a cone

### Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML, Image
%matplotlib inline

### Solution of the ecuations

In [5]:
from scipy.constants import g
from dif_eq_solver import DiffEqSolver


def derivates(t, y):
    '''
    Here we compute dy/dt. As the
    problem corresponds to a particle
    moving on the surface of a cone,
    the variables describing the position
    of the particle are z and theta (3D
    polar coordinates)

    '''

    z_acc = (y[0] * y[3]**2 * tan_alpha**2 - g) / (1 + tan_alpha**2)
    theta_acc = - 2 * y[2] * y[3] / y[0]

    return np.array([y[2], y[3], z_acc, theta_acc])

def calc_energy(solution, time_steps):
    energy = np.empty((len(solution), 4))
    energy[:, 0] = time_steps
    T = 0.5 * m * solution[:, 2] ** 2 * (1 + tan_alpha ** 2) + \
        0.5 * m * solution[:, 0] ** 2 * solution[:, 3] ** 2 * \
        tan_alpha ** 2
    V = - m * g * solution[:, 0]
    energy[:, 1] = T
    energy[:, 2] = V
    energy[:, 3] = T + V
    np.savetxt('cone_energy.txt', energy)
    return energy

def to_cartesian(y):
    '''
    If the solution is a SolutionObject, then the data is in a
    pandas dataframe object.

    '''
    _z = y[0]
    _x = np.cos(y[1]) * _z * tan_alpha
    _y = np.sin(y[1]) * _z * tan_alpha
    return _x, _y, _z

# Constants of the problem
alpha = 15
m, tan_alpha = 1.0, np.tan(alpha * np.pi / 180.0)

# Initial conditions
z0, theta0, z0_vel, theta0_vel = 1.5, 0., 0., np.pi * 2
ti, tf = 0.0, 10
h = .01
initial_conditions = np.array([z0, theta0, z0_vel, theta0_vel])

# Calculate the solution
cone_eqs_solver = DiffEqSolver(derivates, method='rk4')
solution = cone_eqs_solver.solve_eqs(initial_conditions, ti, tf, h, save=True, file_name='cone_solution_polar.txt')

### Representation

In [ ]:
x, y, z = to_cartesian(solution)
ax_plot = solution.fast_plot(x, y, z)
ax_plot.set_xticks(np.linspace(min(x), max(x), 4))
ax_plot.set_yticks(np.linspace(min(x), max(x), 4))

### Animation

In [18]:
rc('animation', html='html5', writer='imagemagick')

In [ ]:
#Be sure to execute the previous 'Representation' code above to define x, y, z!
t = cone_eqs_solver.time_steps

# Attaching 3D axis to the figure
fig = plt.figure(figsize=(10, 10))
ax = p3.Axes3D(fig)

annotate = ax.text(min(x), min(y), max(z), 't(s) = {}'.format(t[0]), size=20)
num_frames = 500
def update_lines(num, dataLines, lines):
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num * len(x) // num_frames])
        line.set_3d_properties(data[2, :num * len(x) // num_frames])
    ax.view_init(azim=num / num_frames * 80)
    annotate.set_text('t(s) = {}'.format(round(t[num * len(t) // num_frames], 1)))
    return lines


# Lines of random 3-D lines
path = np.array([x, y, z])
data = [path]

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]

# Setting the axes properties
ax.set_xlim3d([min(x), max(x)])
ax.set_xlabel('X (m)')

ax.set_ylim3d([min(y), max(y)])
ax.set_ylabel('Y (m)')

ax.set_zlim3d([0, max(z)])
ax.set_zlabel('Z (m)')

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, frames=num_frames, fargs=(data, lines),
                                   interval=50, blit=False)
line_ani.save('particle_cone_3.gif', writer = "imagemagick", fps=50, dpi=40)

Image('particle_cone_3.gif')